In [1]:
from nav2_simple_commander.robot_navigator import BasicNavigator  # BasicNavigator 클래스 임포트
import rclpy  # ROS2 Python 클라이언트 라이브러리 임포트

rclpy.init()  # ROS2 노드 초기화
nav = BasicNavigator()  # BasicNavigator 인스턴스 생성


[WARN] [1753369600.646655961] [rcl]: ROS_LOCALHOST_ONLY is deprecated but still honored if it is enabled. Use ROS_AUTOMATIC_DISCOVERY_RANGE and ROS_STATIC_PEERS instead.
[WARN] [1753369600.646677531] [rcl]: 'localhost_only' is enabled, 'automatic_discovery_range' and 'static_peers' will be ignored.


In [2]:
nav.waitUntilNav2Active()

[INFO] [1753369616.185619365] [basic_navigator]: Nav2 is ready for use!


# 로봇 각도 변환 함수

In [3]:
import math  # 수학 함수 사용을 위한 모듈
import tf_transformations  # 오일러 → 쿼터니언 변환용 모듈

def get_quaternion_from_yaw(yaw_degrees):  # yaw 각도로부터 쿼터니언 생성
    yaw_radians = math.radians(yaw_degrees)  # 도(degree)를 라디안(radian)으로 변환
    quaternion = tf_transformations.quaternion_from_euler(0, 0, yaw_radians)  # Z축 회전만 적용
    return quaternion  # 쿼터니언 반환


# 초기 위치 설정

In [4]:
from geometry_msgs.msg import PoseStamped  # 위치 및 자세 정보를 담는 메시지 타입 임포트

initial_yaw = 0  # 초기 yaw 각도 설정
q = get_quaternion_from_yaw(initial_yaw)  # yaw → quaternion 변환

initial_pose = PoseStamped()  # PoseStamped 메시지 생성
initial_pose.header.frame_id = 'map'  # 참조 좌표계 설정
initial_pose.header.stamp = nav.get_clock().now().to_msg()  # 현재 시간 설정

initial_pose.pose.position.x = 0.0  # X 좌표
initial_pose.pose.position.y = 0.0  # Y 좌표
initial_pose.pose.position.z = 0.0  # Z 좌표

initial_pose.pose.orientation.x = q[0]  # 쿼터니언 X
initial_pose.pose.orientation.y = q[1]  # 쿼터니언 Y
initial_pose.pose.orientation.z = q[2]  # 쿼터니언 Z
initial_pose.pose.orientation.w = q[3]  # 쿼터니언 W

nav.setInitialPose(initial_pose)  # 내비게이터에 초기 위치 설정


[INFO] [1753370054.637313359] [basic_navigator]: Publishing Initial Pose


# amcl 토픽 구독 노드

In [5]:
sub_amcl = rclpy.create_node('sub_amcl')  # 'sub_amcl'이라는 이름의 노드 생성

def amcl_callback(msg):  # AMCL 위치 콜백 함수 정의
    print("===")  # 구분자 출력
    print(msg.pose.pose.position)  # 위치 정보 출력
    print(msg.pose.pose.orientation)  # 방향(quaternion) 정보 출력

from geometry_msgs.msg import PoseWithCovarianceStamped  # AMCL 위치 메시지 타입 임포트

sub_amcl.create_subscription(  # subscriber 생성
    PoseWithCovarianceStamped,  # 메시지 타입
    'amcl_pose',                # 구독할 토픽 이름
    amcl_callback,              # 콜백 함수
    10                          # 메시지 큐 크기
)


In [6]:
from geometry_msgs.msg import PoseStamped  # 목표 위치 설정용 메시지 타입 임포트

goal_yaw = 0  # 목표 yaw 각도를 180도로 설정 (후방을 바라보게 함)
q = get_quaternion_from_yaw(goal_yaw)  # yaw → quaternion 변환

goal_pose = PoseStamped()  # PoseStamped 메시지 생성
goal_pose.header.frame_id = 'map'  # 기준 좌표계를 'map'으로 설정
goal_pose.header.stamp = nav.get_clock().now().to_msg()  # 현재 시간 설정

goal_pose.pose.position.x = 0.0  # 목표 위치 x 좌표
goal_pose.pose.position.y = 1.0  # 목표 위치 y 좌표
goal_pose.pose.position.z = 0.0  # 목표 위치 z 좌표

goal_pose.pose.orientation.x = q[0]  # quaternion x
goal_pose.pose.orientation.y = q[1]  # quaternion y
goal_pose.pose.orientation.z = q[2]  # quaternion z
goal_pose.pose.orientation.w = q[3]  # quaternion w

nav.goToPose(goal_pose)

[INFO] [1753370105.873719785] [basic_navigator]: Navigating to goal: 0.0 1.0...


True

# 상황 피드백 받기

In [17]:
from rclpy.duration import Duration  # 시간 비교를 위한 Duration 클래스 임포트
from IPython.display import clear_output  # Jupyter 환경에서 출력 정리용 함수

while not nav.isTaskComplete():  # 작업이 완료될 때까지 반복
    clear_output(wait=True)  # 이전 출력 지우기 (Jupyter 환경 전용)
    rclpy.spin_once(sub_amcl, timeout_sec=1.0)  # AMCL 콜백 1초 대기하며 실행

    feedback = nav.getFeedback()  # 현재 내비게이션 피드백 받기
    print('===')  # 구분선 출력
    print('Distance remaining: ' + '{:.2f}'.format(  # 남은 거리 출력
        feedback.distance_remaining) + ' meters.')

    if Duration.from_msg(feedback.navigation_time) > Duration(seconds=30.0):  # 30초 이상 경과 시
        nav.cancelTask()  # 작업 취소


===
geometry_msgs.msg.Point(x=0.14093165886686, y=0.7935431040698563, z=0.0)
geometry_msgs.msg.Quaternion(x=0.0, y=0.0, z=0.023279307207261766, w=0.9997290002075312)
===
Distance remaining: 0.26 meters.


# 결과 확인용

In [18]:
from nav2_simple_commander.robot_navigator import TaskResult  # 내비게이션 결과 상태 enum 임포트

result = nav.getResult()  # 내비게이션 결과 상태 가져오기

if result == TaskResult.SUCCEEDED:  # 성공적으로 도착한 경우
    print('Goal succeeded!')

elif result == TaskResult.CANCELED:  # 작업이 취소된 경우
    print('Goal was canceled!')

elif result == TaskResult.FAILED:  # 실패한 경우
    print('Goal failed!')

else:  # 알 수 없는 상태일 경우
    print('Goal has an invalid return status!')


Goal succeeded!


# Waypoint 여러개 만들기

In [15]:
from geometry_msgs.msg import PoseStamped  # 목표 위치 설정용 메시지 타입 임포트


goal_pose_list = []  # 여러 개의 목표 위치를 담을 리스트 생성

goal_yaw1 = 0  # 첫 번째 목표의 yaw 각도 (정면)
q1 = get_quaternion_from_yaw(goal_yaw1)  # yaw → quaternion 변환

goal_pose1 = PoseStamped()  # 첫 번째 목표 Pose 생성
goal_pose1.header.frame_id = 'map'  # 기준 좌표계를 'map'으로 설정
goal_pose1.header.stamp = nav.get_clock().now().to_msg()  # 현재 시간으로 타임스탬프 설정
goal_pose1.pose.position.x = 1.31  # x 좌표
goal_pose1.pose.position.y = 1.0  # y 좌표
goal_pose1.pose.orientation.x = q1[0]  # quaternion x
goal_pose1.pose.orientation.y = q1[1]  # quaternion y
goal_pose1.pose.orientation.z = q1[2]  # quaternion z
goal_pose1.pose.orientation.w = q1[3]  # quaternion w
goal_pose_list.append(goal_pose1)  # 리스트에 추가

goal_yaw2 = 180  # 두 번째 목표의 yaw 각도 (뒤쪽)
q2 = get_quaternion_from_yaw(goal_yaw2)  # yaw → quaternion 변환

goal_pose2 = PoseStamped()  # 두 번째 목표 Pose 생성
goal_pose2.header.frame_id = 'map'
goal_pose2.header.stamp = nav.get_clock().now().to_msg()
goal_pose2.pose.position.x = -1.34
goal_pose2.pose.position.y = 1.0
goal_pose2.pose.orientation.x = q2[0]
goal_pose2.pose.orientation.y = q2[1]
goal_pose2.pose.orientation.z = q2[2]
goal_pose2.pose.orientation.w = q2[3]
goal_pose_list.append(goal_pose2)

goal_yaw3 = 45  # 세 번째 목표의 yaw 각도 (대각선)
q3 = get_quaternion_from_yaw(goal_yaw3)

goal_pose3 = PoseStamped()
goal_pose3.header.frame_id = 'map'
goal_pose3.header.stamp = nav.get_clock().now().to_msg()
goal_pose3.pose.position.x = -1.39
goal_pose3.pose.position.y = -0.36
goal_pose3.pose.orientation.x = q3[0]
goal_pose3.pose.orientation.y = q3[1]
goal_pose3.pose.orientation.z = q3[2]
goal_pose3.pose.orientation.w = q3[3]
goal_pose_list.append(goal_pose3)


In [16]:
nav_start = nav.get_clock().now()  # 경로 추적 시작 시각 저장
nav.followWaypoints(goal_pose_list)  # 여러 목표 지점을 따라 이동 명령 실행


[INFO] [1753359056.235398893] [basic_navigator]: Following 3 goals....


True

# 진행상황 살펴보기

In [17]:
from rclpy.duration import Duration  # 시간 비교를 위한 Duration 클래스 임포트
from IPython.display import clear_output  # Jupyter 환경에서 출력 정리용 함수

while not nav.isTaskComplete():  # 작업이 완료될 때까지 반복
    clear_output(wait=True)  # 이전 출력 제거 (Jupyter 환경 전용)
    rclpy.spin_once(sub_amcl, timeout_sec=1.0)  # AMCL subscriber 콜백 1초간 실행

    feedback = nav.getFeedback()  # 현재 내비게이션 피드백 정보 가져오기

    print("===")  # 출력 구분선
    print(  # 현재 수행 중인 웨이포인트 인덱스와 전체 개수 출력
        'Executing current waypoint: '
        + str(feedback.current_waypoint + 1)  # 현재 인덱스 (1부터 시작하는 표현)
        + '/' + str(len(goal_pose_list))  # 전체 웨이포인트 수
    )

    now = nav.get_clock().now()  # 현재 시간
    if now - nav_start > Duration(seconds=300.0):  # 5분(300초) 초과 시
        nav.cancelTask()  # 내비게이션 작업 취소


===
Executing current waypoint: 3/3


In [3]:
nav.lifecycleShutdown()
rclpy.shutdown()

[INFO] [1753359325.163934274] [basic_navigator]: Shutting down lifecycle nodes based on lifecycle_manager.
[INFO] [1753359325.164946630] [basic_navigator]: Shutting down /lifecycle_manager_localization/manage_nodes
[INFO] [1753359326.166690752] [basic_navigator]: /lifecycle_manager_localization/manage_nodes service not available, waiting...
[INFO] [1753359327.167659625] [basic_navigator]: /lifecycle_manager_localization/manage_nodes service not available, waiting...
[INFO] [1753359328.168697650] [basic_navigator]: /lifecycle_manager_localization/manage_nodes service not available, waiting...
[INFO] [1753359329.169706751] [basic_navigator]: /lifecycle_manager_localization/manage_nodes service not available, waiting...
[INFO] [1753359330.170766352] [basic_navigator]: /lifecycle_manager_localization/manage_nodes service not available, waiting...
[INFO] [1753359331.171743276] [basic_navigator]: /lifecycle_manager_localization/manage_nodes service not available, waiting...
[INFO] [175335933

KeyboardInterrupt: 